Before EDA let's reroll a bit:

In `1_load_data__understand.ipynb` notebook in `\notebooks\testing` we;
* load data
* Set index to `PassengerId` feature
* Get some kind of understanding from bird view on dataset
  * We find Null values and Decided actions such
    * `Age` attribute has *177* NaN
      * Fill up with a regression model (not decided yet)
    * `Cabin` attribute has *687* NaN
      * First we will extract `CabinType` then delete `Cabin` feature
      * Second We will fill up with a classification model (not decided yet)
    * `Embarked` attribute has *2* NaN
      * Simple Classification will enough for filling
* Detecting Outliers;
  * We will search for `Age` and `Fare` attributes for outliers
* We will create some new features;
  * From `Age` --> Create `AgeBucket` --> Delete `Age` If good for model
  * From `Name` --> Create `Title` and `FamilySize` --> Delete `Name`
  * From `SibSp` and `Parch` --> Create `RelativesOnBoard` --> Delete `SibSp` and `Parch` If good for model
  * From `Ticket` --> Create `DoesHaveSameTicket` --> Delete `Ticket`
  * From `Cabin` --> Create `CabinType` --> Delete `Cabin`
* We found out skewness
  * `Fare` feature has positive skewness; we will handle with **log transformation**
  * `Age` feature has possible skewness; bu will ignore it because we won't use this feature directly
* Encoding;
  * `Sex`, `Embarked`, `DoesHaveSameTicket`(We will create), `Title`(We will create) and `CabinType`(We will create) will use **OneHotEncode**
  * `Pclass` will use **OrdinalEncode**

Let's Create data preprocessing guideline:

1. Fill up NaN values
2. Search for Outliers
3. Create Features
4. Handle Skewness if has it
5. Encode

| Feature | Steps |
| ------- | :--: |
| `Pclass` | 5 |
| `Name` | 3 |
| `Sex` | 5 |
| `Age` | 1, 2, 3, 4 (if not deleted) |
| `SibSp` | 3 |
| `Parch` | 3 |
| `Ticket` | 3 |
| `Fare` | 2, 4 |
| `Cabin` | 3 |
| `Embarked` | 1, 5 |

**Feature Creation And Deletions**
| Feature From | New Feature | New Feature Steps | Deletion of Old one? |
| ------------ | ----------- | :---------------: | :-------------: |
| `Name` | `Title` | 5, 2, 5 | Yes |
| `Name` | `FamilySize` | | Yes |
| `Age` | `AgeBucket` | | Decide by score |
| `SibSp` and `Parch` | `RelativesOnBoard` | | Decide by score |
| `Ticket` | `DoesHaveSameTicket` | 5 | Yes |
| `Cabin` | `CabinType` | 5 | Yes |

* `Name`, `Ticket` and `Cabin` features after creating new features will **DELETED**.
* `Age`, `SibSp` and `Parch` features after creating new features, we will see impact on a model then decided for deleted or not.

**Note:**
I did not include scaling for now.

These inferences are after first glance from dataset; can add to it or reduce it. I want to write now before EDA, because I will much more deep info after EDA, then I can review my first view and determine my actions accordingly.

---

# EDA